In [ ]:
#add all imports
import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
import os

%matplotlib inline

# Camera Calibration

In [ ]:
def findImgObjPts(img_dir, nx, ny):
    '''
    This functions takes input image directory and the no. of inner corners in x & y direction, 
    reads images, looks for chessboard corners for a chessboard size of (nx * ny)
    and returns the objectpoints and imagepoints if corners are found
    '''
    images = glob.glob(img_dir + '*.jpg')
    
    objpts = []
    imgpts = []
    
    objp = np.zeros((nx * ny,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)
    
    for image in images:
        img = cv2.imread(image)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)

        if ret:
            imgpts.append(corners)
            objpts.append(objp)
    
    return objpts, imgpts

In [ ]:
def undistort(img, objpts, imgpts):
    '''
    Takes in a distorted input image, object points and image points and returns the undistorted image
    '''
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpts, imgpts, img.shape[1::-1], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    
    return undist

### Test the calibration and undistort functions using the test pipeline below

In [ ]:
# Specify the input images directory and extract the objpts and imgpts 
input_dir = './camera_cal/'
x_corners = 9
y_corners = 6
objpts, imgpts = findImgObjPts(input_dir, x_corners, y_corners)

# Test the undistort image function on a test image
#img = plt.imread(input_dir + 'calibration1.jpg')
img = plt.imread('./test_images/test1.jpg')
undistorted_img = undistort(img, objpts, imgpts)
fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=20)
ax2.imshow(undistorted_img)
ax2.set_title('Undistorted Image', fontsize=20)

## Helper functions for color conversion

In [ ]:
def gray_conv(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
def RGBtoHLS(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2HLS)

## Create a thresholded binary image

In [ ]:
def abs_sobel_thresh(img, orient='x', kernel_size=3, threshold=(0, 255)):
    
    gray = gray_conv(img)
    
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=kernel_size)
        
    elif orient == 'y':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=kernel_size)
        
    abs_sobel = np.absolute(sobel)
    scaled_sobel = np.uint8(255 * abs_sobel/np.max(abs_sobel))
    
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= threshold[0]) & (scaled_sobel <= threshold[1])] = 1
    
    return grad_binary


def mag_threshold(img, kernel_size = 3, mag_thresh = (0, 255)):
    
    gray = gray_conv(img)
    
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=kernel_size)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=kernel_size)
    
    magnitude = np.sqrt((sobelx**2)  + (sobely**2))
    #abs_magnitude = np.abs(magnitude)
    scaled_mag = np.uint8(255 * magnitude/np.max(magnitude))
    
    mag_binary = np.zeros_like(scaled_mag)
    mag_binary[(scaled_mag >= mag_thresh[0]) & (scaled_mag <= mag_thresh[1])] = 1
    
    return mag_binary


def dir_threshold(img, kernel_size = 3, dir_thresh = (0, np.pi/2)):
    
    gray = gray_conv(img)
    
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=kernel_size)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=kernel_size)
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    
    direc = np.absolute(np.arctan2(abs_sobely, abs_sobelx))
    dir_binary = np.zeros_like(direc)
    dir_binary[(direc >= dir_thresh[0]) & (direc <= dir_thresh[1])] = 1
    
    return dir_binary


def combi_threshold(img):
    
    sobelx_binary = abs_sobel_thresh(img, 'x', threshold=(10,200))
    sobely_binary = abs_sobel_thresh(img, 'y', threshold=(10,200))
    
    mag_binary = mag_threshold(img, mag_thresh=(10,200))
    dir_binary = dir_threshold(img, dir_thresh=(np.pi/6, np.pi/2))
    
    combined_binary = ((sobelx_binary == 1) & (dir_binary == 1))
    
    R_channel = img[:,:,0]
    G_channel = img[:,:,1]
    
    R_G_binary = (R_channel > 150) & (G_channel > 150)
    
    #convert to HLS color space and extract s_channel using helper functions
    hls_img = RGBtoHLS(img)
    
    s_channel = hls_img[:,:,2]
    s_ch_binary = (s_channel >= 100) & (s_channel <= 255)
    
    l_channel = hls_img[:,:,1]
    l_ch_binary = (l_channel >= 120) & (l_channel <= 255)
    
    final_binary = np.zeros_like(R_channel)
    final_binary[(R_G_binary & l_ch_binary) & (s_ch_binary | combined_binary)] = 1
    
    return final_binary

## Perspective transform on lane lines

In [ ]:
def perspective_transform(img):
    
    img_size = (img.shape[1], img.shape[0])
    
    src = np.float32([[568, 468], [715, 468], [1040, 680], [270, 680]])
    dest = np.float32([[200, 0], [1000, 0], [1000, 680], [200, 680]])
    
    M = cv2.getPerspectiveTransform(src, dest)
    
    Minv = cv2.getPerspectiveTransform(dest, src)
    
    warped_img = cv2.warpPerspective(img, M, img_size, flags = cv2.INTER_LINEAR)
    
    return warped_img, M, Minv

## Find lane lines 

In [ ]:
def locate_lane_pixels(binary_warped):

    #Number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 200
    
    # Take a histogram of the bottom half of the warped binary image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:, :], axis=0)
    
    midpt = np.int(len(histogram)//2)
    leftx_base = np.argmax(histogram[:midpt])
    rightx_base = np.argmax(histogram[midpt:]) + midpt
    
    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    
    # Identify the x and y positions of all nonzero (i.e. activated) pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroY = np.array(nonzero[0])
    nonzeroX = np.array(nonzero[1])
    
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    height = binary_warped.shape[0]
    
    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_top = height - (window+1)*window_height
        win_bottom = height - window*window_height
        
        win_xleft_low = leftx_current - margin     #left x value for left window
        win_xleft_high = leftx_current + margin    #right x value for left window
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroY >= win_top) & (nonzeroY < win_bottom)\
                          & (nonzeroX >= win_xleft_low) & (nonzeroX < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroY >= win_top) & (nonzeroY < win_bottom)\
                           & (nonzeroX >= win_xright_low) & (nonzeroX < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzeroX[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzeroX[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftX = nonzeroX[left_lane_inds]
    leftY = nonzeroY[left_lane_inds] 
    rightX = nonzeroX[right_lane_inds]
    rightY = nonzeroY[right_lane_inds] 
    
    left_fit = np.polyfit(leftY, leftX, 2)
    right_fit = np.polyfit(rightY, rightX, 2)
    
#     if (len(leftX) == 0):
#         left_fit = None
#     else:
#         left_fit = np.polyfit(leftY, leftX, 2)
#     # For right lane
#     if (len(rightX) == 0):
#         right_fit = None
#     else:
#         right_fit = np.polyfit(rightY, rightX, 2)
    
    return left_fit, right_fit, left_lane_inds, right_lane_inds, nonzeroX, nonzeroY
    

def fetch_lines(left_fit, right_fit, binary_warped):
    
    # Generate x and y values for plotting
    plotY = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    left_fitX = left_fit[0]*plotY**2 + left_fit[1]*plotY + left_fit[2]
    right_fitX = right_fit[0]*plotY**2 + right_fit[1]*plotY + right_fit[2]
    return left_fitX, right_fitX, plotY


def extend_lines(left_fit, right_fit, binary_warped):
    
    margin = 100
    nonzero = binary_warped.nonzero()
    nonzeroY = np.array(nonzero[0])
    nonzeroX = np.array(nonzero[1])
    
    left_lane_inds = ((nonzeroX > (left_fit[0]*(nonzeroY**2) + left_fit[1]*nonzeroY + left_fit[2] - margin)) 
                      & (nonzeroX < (left_fit[0]*(nonzeroY**2) + left_fit[1]*nonzeroY + left_fit[2] + margin))) 
    right_lane_inds = ((nonzeroX > (right_fit[0]*(nonzeroY**2) + right_fit[1]*nonzeroY + right_fit[2] - margin)) 
                       & (nonzeroX < (right_fit[0]*(nonzeroY**2) + right_fit[1]*nonzeroY + right_fit[2] + margin)))      
    
    leftX = nonzeroX[left_lane_inds]
    leftY = nonzeroY[left_lane_inds] 
    rightX = nonzeroX[right_lane_inds]
    rightY = nonzeroY[right_lane_inds]
    left_fit, right_fit = (None, None)
    
    # Fit a second order polynomial to each
    if len(leftX) != 0:
        left_fit = np.polyfit(leftY, leftX, 2)
    if len(rightX) != 0:
        right_fit = np.polyfit(rightY, rightX, 2)
    return left_fit, right_fit

## Radius of curvature

In [ ]:
def curvature_radius(left_fitX, right_fitX, plotY):
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    left_fitX_m = left_fitX * xm_per_pix
    right_fitX_m = right_fitX * xm_per_pix
    plotY_m = plotY * ym_per_pix
    
    y_eval = np.max(plotY_m)
    leftFit = np.polyfit(plotY_m, left_fitX_m, 2)
    rightFit = np.polyfit(plotY_m, right_fitX_m, 2)
    
    leftRadius = ((1 + (2 * leftFit[0] * y_eval + leftFit[1])**2)**1.5) / np.absolute(2 * leftFit[0])
    rightRadius = ((1 + (2 * rightFit[0] * y_eval + rightFit[1])**2)**1.5) / np.absolute(2 * rightFit[0])
    
    avgRadius = (leftRadius + rightRadius)/2
    
    return avgRadius

## Lane values

In [ ]:
def fetch_dist_offset(left_fitX, right_fitX, plotY):
    
    xm_per_pix = 3.7/480
    
    avg_dist = np.mean(right_fitX - left_fitX) * xm_per_pix
    
    lane_center = (left_fitX[-1] + right_fitX[-1])//2
    
    car_center = 640
    
    center_offset = (lane_center - car_center) * xm_per_pix
    
    return avg_dist, center_offset


def lane_check(binary_warped, left_fit, right_fit):
    
    left_fitX, right_fitX, plotY = fetch_lines(left_fit, right_fit, binary_warped)
    
    avg_dist, center_offset = fetch_dist_offset(left_fitX, right_fitX, plotY)
    
    if avg_dist < 2.5 or avg_dist > 5:
        return False
    if center_offset > 2:
        return False
    
    radius = curvature_radius(left_fitX, right_fitX, plotY)
    if radius < 30:
        return False
    if radius < 10000 and (left_fit[0] * right_fit[0]) < 0:
        return False

    return True

## Draw on image

In [ ]:
def draw_lanes_text(undist, binary_warped, left_fit, right_fit, Minv, curvature, center_offset):
    
    left_fitX, right_fitX, plotY = fetch_lines(left_fit, right_fit, binary_warped)
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitX, plotY]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitX, plotY])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (undist.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    
    
    ## Draw text on image

    text = 'Lane curvature: ' + '{:.0f}'.format(curvature) + 'm'
    cv2.putText(result, text, (50, 50), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
    position = "left" if center_offset < 0 else "right"
    abs_center_offset = abs(center_offset)
    text = 'Vehicle posn: {:.2f}'.format(abs_center_offset) + 'm ' + position + ' from center'
    cv2.putText(result, text, (50, 100), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    return result

## Final Video Pipeline

In [ ]:
class Lane():
    def __init__(self):
        self.last_left = None
        self.last_right = None
        self.left_fit = None
        self.right_fit = None
        
lane = Lane()

def video_processing_pipeline(img):
   
    undistored_img = undistort(img, objpts, imgpts)

    binary_img = combi_threshold(undistored_img)

    binary_warped, M, Minv = perspective_transform(binary_img)

    if lane.last_left == None or lane.last_right == None:
        lane.left_fit, lane.right_fit, _, _, _, _ = locate_lane_pixels(binary_warped)
    else:
        lane.left_fit, lane.right_fit = extend_lines(lane.last_left, lane.last_right, binary_warped)
        if lane.left_fit is not None and lane.right_fit is not None:
            if not lane_check(binary_warped, lane.left_fit, lane.right_fit):
                lane.left_fit, lane.right_fit, _, _, _, _ = locate_lane_pixels(binary_warped)

    lane.last_left = lane.left_fit
    lane.last_right = lane.right_fit

    if lane.last_left is not None and lane.last_right is not None:
        leftX, rightX, plotY = fetch_lines(lane.last_left, lane.last_right, binary_warped)
        avg_dist, center_offset = fetch_dist_offset(leftX, rightX, plotY)
        radius = curvature_radius(leftX, rightX, plotY)
        result = draw_lanes_text(undistored_img, binary_warped, lane.last_left, lane.last_right, Minv, radius, center_offset)
        return result
    else:
        return undistored_img

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML


project_output = 'project_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
# clip1 = VideoFileClip("project_video.mp4").subclip(0,5)
clip1 = VideoFileClip("project_video.mp4")
project_clip = clip1.fl_image(video_processing_pipeline) #NOTE: this function expects color images!!
%time project_clip.write_videofile(project_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_output))